# Introducció a las Xarxes neuronals convolucionals

**Assignatura**: Models d'intel·ligència artificial

**Professor** : Ramon Mateo Navarro
<p style="text-align: justify;">

En aquest notebook farem una introducció a les xarxes neuronals convolucionals. Aprendrem les bases per crear la nostra primera xarxa neuronal convolucional fent servir diferents datasets i explorarem els problemes que presenta grans volums de dades.
</p>

## Recordem:

<p style="text-align: justify;">

Les xarxes neuronals convolucionals (CNN) utilitzen l'operació matemàtica de convolució. Aquesta operació es realitza amb un kernel, que és una matriu de dimensions n x m, i el conjunt de tots els kernels es denomina filtres.

L'operació de convolució consisteix en calcular el producte puntual entre el kernel i una regió corresponent de l'imatge d'entrada, i després sumar tots els resultats d'aquests productes. Aquest procés es realitza a través de la imatge aplicant el mateix kernel en diferents ubicacions, produint una imatge convolucionada com a resultat.
</p>

![](images_lab3\convolucio.gif)

<p style="text-align: justify;">

Per reduir les dimensions de la imatge resultant en una xarxa neuronal convolucional, es poden aplicar tècniques com el max pooling, l'average pooling i altres tècniques de pooling. Aquestes capes de pooling redueixen la mida espacial (alçada i amplada) de la imatge mantenint només la informació més rellevant. En el cas del max pooling, es conserva el valor màxim de cada regió, mentre que en l'average pooling es calcula el valor promig. Aquestes tècniques ajuden a reduir el nombre de paràmetres a la xarxa i a augmentar la invariància a petites traslaciones en l'entrada.
</p>


![](images_lab3\max_avg.gif)






## Imports

Per aquest exercici farem servir tensorflow i keras.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tensorflow.keras import datasets, layers, models


## Carrega i visualització del dataset

Per aquest primer exercici farem servir el dataset CIFAR10 que conté 60.000 imatges. sis mil imatges per cada classe (en total 10 classes diferents).

Link del dataset: [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)

A recordar:
* Recordem que les imatges es componen de tres canals RGB. Un per cada color. 
* Que normalitzar valors ajuda el model a ser entrenat millor.


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Visualitzem el dataset per veure com son les imatges

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## Definició del model

Anteriorment, ja hem vist com declarar un model amb tensorflow/keras. Farem ara exactament el mateix però fent servir les capes de convolució 2D i les de pooling. 

**Nota**: Les capes de convolució 2D s'utilitzen quan es vol processar una única imatge en cada moment. Per a l'anàlisi de vídeos, que requereix considerar la dimensió temporal a més de les espacials, cal fer servir capes de convolució 3D.

Links: 
* Capa de convolució: [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
* Capa de maxpooling: [MaxPooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPooling2D)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

## Compilació i entrenament del model

Per aquest entrenament farem servir l'optimitzador Adam, com a funció de pèrdua farem servir la Sparse Categorical Cross Entropy que ja s'ha explicat anteriorment.

Fixeu-vos que afegim un dataset de validació que són les imatges de test.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, batch_size=16, 
                    validation_data=(test_images, test_labels))

## Evaluant el model

Anem analitzar l'entranment del model:

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)

## Predint noves imatges

In [ ]:
idx = 3
image = np.expand_dims(test_images[idx], axis=0)
prediction = np.argmax(model.predict(image))
print("Value predicted", class_names[prediction], "\nValue expected:", class_names[test_labels[idx][0]])
plt.figure(figsize=(2,2))
plt.imshow(image[0])
plt.show()

## Tasques:

* Mireu que succeix quan modifiqueu el batchsize així com el número de epochs.
* Proveu d'afegir noves capes.
* Descarregueu aquest dataset i proveu de crear el vostre model de classificació (farem el preprocessament junts): [Pokemon Dataset](https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types)

## Preprocessant el nou dataset

Si observem veurem que tenim la categoria (el tipus i el que volem arribar a predir) en format categòric. Recordem que un model no sap predir variables categòriques sinó numèriques. Per tant, haurem de transformar aquestes dades a valors numèrics.

In [ ]:
data = pd.read_csv('dataset lab3\pokemon.csv')
data.head()

Anem primer de tot eliminar les columnes que no volem, en aquest exercici ens centrarem només a intentar predir el tipus 1 del pokemon.

In [ ]:
data = data.drop(['Type2', 'Evolution'], axis='columns')
data.head()

Anem a transformar ara type1 en valors numèrics

In [ ]:
tipus = pd.get_dummies(data.Type1)
tipus.head(5)

Fixeu-vos com ara totes tenen tot a false menys una columna que està a true que representa el valor que són.


## Carregant imatges

Ara toca carregar les imatges, fixeu-vos que tots els Pokémon tenen el seu nom ben indicat i el tipus. El que podem fer és per cada fila del dataset carregar la imatge corresponent.

In [ ]:
total_images = list()

for row in data.iterrows():
    image = cv2.imread('dataset_lab3\\images\\' + row[1][0] + '.png')
    image = cv2.resize(image, (64,64))
    total_images.append(image)

total_images = np.array(total_images)